### 임베딩 객체 생성(쿼리 임베딩)

In [ ]:

from dotenv import load_dotenv
import os
from langchain_openai import AzureOpenAIEmbeddings
from langchain_community.vectorstores import AzureSearch

load_dotenv("E:/work/MS_project_2/code/.env")

embedding_api_key = os.getenv('Embedding_API_KEY')
embedding_endpoint = os.getenv('Embedding_ENDPOINT')

embedding_api_version = os.getenv('embedding_api_version')
embedding_deployment = os.getenv('embedding_deployment')

os.environ.pop("OPENAI_API_BASE", None)
os.environ.pop("BASE_URL", None)

ai_search_endpoint = os.getenv("add_new_index_Search_ENDPOINT")
ai_search_api_key = os.getenv('AI_Search_API_KEY')



In [21]:
print("실행디렉토리:", os.getcwd())
print("Embedding_API_KEY:", os.getenv("Embedding_API_KEY"))
print(ai_search_endpoint)

실행디렉토리: e:\work\MS_project_2\code\테이블처리o
Embedding_API_KEY: 11UsWjDAHdmUd88UAtmuBciucqMX0L9Qafi0Mzg0a9Pw5hIl90QkJQQJ99BDACfhMk5XJ3w3AAABACOGsig4
https://6b034-ai-search.search.windows.net/indexes/add_new_index/docs/search?api-version=2023-10-01-Preview


In [22]:

# ✅ 임베딩 객체 (쿼리를 임베딩하는 데 사용)
embedding = AzureOpenAIEmbeddings(
    api_key = embedding_api_key,
    azure_endpoint = embedding_endpoint,
    model = embedding_deployment,  
    openai_api_version = embedding_api_version
)

### source값 가져오기

In [52]:
import requests

def request_ai_search(query: str, source_filter: str = None, k: int = 5) -> list:
    headers = {
        "Content-Type": "application/json",
        "api-key": ai_search_api_key
    }

    query_vector = embedding.embed_query(query)

    body = {
        "search": query,  # 🔥 키워드 검색 추가!
        "vectorQueries": [
            {
                "kind": "vector",
                "vector": query_vector,
                "fields": "embedding",
                "k": k
            }
        ]
    }

    if source_filter:
        cleaned_source = source_filter.replace(".pdf", "")
        body["filter"] = f"source eq '{cleaned_source}'"

    response = requests.post(ai_search_endpoint, headers=headers, json=body)

    if response.status_code != 200:
        print(f"❌ 검색 실패: {response.status_code}")
        print(response.text)
        return []

    response_json = response.json()
    results = [
        {
            "content": item["content"],
            "source": item.get("source", ""),
            "score": item.get("@search.score", 0)
        }
        for item in response_json["value"]
    ]

    return results


# 예시 실행
query = "입주 후 혼인"
source = "(대전충남)25년1차청년매입임대_표준입주자모집공고문.pdf"  # 확장자는 제거되어 비교됨

results = request_ai_search(query, source_filter=source, k=5)

for i, result in enumerate(results):
    print(f"\n🔍 Top {i+1} (score: {result['score']:.4f})")
    print(f"출처: {result['source']}")
    print(result["content"])



🔍 Top 1 (score: 0.0328)
출처: (대전충남)25년1차청년매입임대_표준입주자모집공고문
입주자가 주택도시기금의 대출을 받은 경우 임대주택에 입주하기 전까지 대출금을 상환하고 증빙서류를 제출해야 합니다. 단, 해당 은행이 공사 매입임대주택으로 대출 목적물 변경을 승인한 경우 상환이 필요 없으며, 세부사항은 해당 은행과 사전에 상담해야 합니다. 타 임대주택(국민임대, 전세임대 등)은 매입임대주택 입주 전에 임대사업자에게 명도해야 합니다. 입주자가 계약체결 및 입주 이후에도 주택을 소유한 사실이 확인되면 임대차계약은 해지되고 퇴거해야 합니다. 계약체결 후 60일 이내에 입주를 완료해야 합니다.  
수급자, 지원대상 한부모가족, 차상위계층은 자산검증을 받지 않습니다. 첨부된 주택내역의 임대조건은 모집공고일 기준이며, 계약일정에 따라 임대조건이 변경될 수 있습니다.

🔍 Top 2 (score: 0.0311)
출처: (대전충남)25년1차청년매입임대_표준입주자모집공고문
입주자 선정: 모집공고일 현재 관계법령에서 정한 입주 자격을 갖춘 자만 신청 가능하며, 입주 시까지 자격을 유지해야 합니다. 주택 소유 여부 확인 및 판정 기준은 "주택공급에 관한 규칙 제53조"에 따릅니다. 외국인은 신청할 수 없으며, 미성년자는 계약 시 법정대리인 동의서와 인감증명서를 지참해야 합니다. 부모 범위에서 제외되는 경우는 신청자가 이를 입증해야 하며, 국민기초생활보장법 시행령에 따라 실종선고 절차 진행 중인 사람, 가출 또는 행방불명으로 신고된 후 1개월 이상 경과한 사람, 생계 및 주거를 달리한다고 확인된 사람이 이에 해당합니다. 세대원의 실종 또는 별거로 인해 소득·자산 파악이 불가능한 경우는 주민등록표 말소를 확인하여 소득·자산 산정 대상에서 제외됩니다.  
주택개방: 예비입주자 모집공고는 청약 접수 기간 및 계약 체결 전 LH 지역본부의 여건에 맞게 주택 개방 일정이 진행됩니다. 주택 개방 일정은 주택 내역 또는 별도 공지사항으로 안내될 예정입니다.

🔍 Top 3 (

### LLM 연동

In [48]:
import requests
import re

# Azure OpenAI GPT-4o 설정
llm_endpoint = os.getenv('OPENAI_ENDPOINT')
llm_api_key = os.getenv('OPENAI_API_KEY')

def request_gpt(prompt: str) -> str:
    """
    GPT-4o에 프롬프트를 보내고 응답 받기
    """
    headers = {
        'Content-Type': 'application/json',
        'api-key': llm_api_key
    }

    body = {
        "messages": [
            {
                "role": "system",
                "content": "너는 친절하고 정확한 AI 도우미야. 사용자 질문에 문서 기반으로 답해줘."
            },
            {
                "role": "user",
                "content": prompt
            }
        ],
        "temperature": 0.7,
        "top_p": 0.95,
        "max_tokens": 800
    }

    response = requests.post(llm_endpoint, headers=headers, json=body)

    if response.status_code == 200:
        response_json = response.json()
        message = response_json['choices'][0]['message']
        content = message['content']
        # [doc1] 같은 포맷이 있을 경우 사람이 읽기 쉽게 변환
        content = re.sub(r'\[doc(\d+)\]', r'[참조 \1]', content)
        return content
    else:
        print("❌ 요청 실패:", response.status_code, response.text)
        return "⚠️ 오류가 발생했습니다."


In [49]:
def generate_answer_with_rag(query: str, source_filter: str = None, top_k: int = 3) -> str:
    # 🔍 벡터 유사도 기반 검색 수행 (REST API 기반)
    results = request_ai_search(query, source_filter=source_filter, k=top_k)

    # 결과가 없으면 안내
    if not results:
        return "❌ 관련 문서를 찾을 수 없습니다."

    # 🔗 context 구성
    context = "\n\n".join([f"[doc{i+1}]\n{item['content']}" for i, item in enumerate(results)])

    # 🤖 LLM 프롬프트 생성
    prompt = f"""다음은 사용자가 질문한 내용과 관련된 문서 내용이야. 이 문서를 참고해서 질문에 대해 정확하고 구체적으로 답변해줘.
                그리고 어떤 문서에서 찾았는지도 출처도 알려줘
[사용자 질문]
{query}

[참고 문서]
{context}

답변:"""

    return request_gpt(prompt)


In [53]:
query = "나 대학생인데 혜택있냐?"
source = "(대전충남)25년1차청년매입임대_표준입주자모집공고문.pdf"

answer = generate_answer_with_rag(query, source_filter=source, top_k=5)
print("🧠 GPT 응답:\n", answer)


🧠 GPT 응답:
 대학생이라면 "청년 매입임대주택"이라는 혜택을 받을 수 있습니다. 이 주택은 한국토지주택공사(LH)가 19세부터 39세 사이의 청년, 대학생, 취업준비생을 대상으로 시중 시세의 40~50% 수준으로 임대하는 주택입니다. 대학생은 재학 중이거나 입학·복학 예정자도 신청할 수 있습니다. 다만, 신청 후 재학증명서나 입학·복학 관련 증명서를 제출해야 하며, 기한 내 제출하지 않으면 신청이 취소될 수 있습니다. 

청년 매입임대주택에 대한 자세한 정보는 LH 마이홈센터(1600-1004)에서 상담받을 수 있으며, 모집공고를 통해 세부 내용을 확인해야 합니다.

관련 내용은 아래 문서에서 확인되었습니다:
- [참조 3]: 대학생 포함 청년 매입임대주택 자격 조건
- [참조 6]: 청년 매입임대주택에 대한 설명
- [참조 7]: 대학생의 신청 절차 및 제출 서류

추가로 궁금한 점이나 신청 방법에 대해 알고 싶다면 질문 주세요!
